# Resource Server Token Validation

This notebook models how a Resource Server enforces authorization
using OAuth access tokens.

Goals:
- Understand what a Resource Server trusts
- Learn what it must NEVER do
- See how scopes are enforced


## What a Resource Server Sees

A Resource Server receives:

Authorization: Bearer <access_token>

It does NOT receive:
- client_id
- redirect_uri
- PKCE data
- CIMD metadata

All of that already happened upstream.


## Resource Server Mindset

A Resource Server:
- Does NOT authenticate clients
- Does NOT fetch CIMD metadata
- Does NOT know client_id
- Does NOT run OAuth flows

It ONLY:
- Validates tokens
- Enforces scopes
- Protects resources


In [12]:
# Simulated token store (issued by Authorization Server)
ISSUED_TOKENS = {
    "token-abc": {
        "issuer": "https://auth.example.com",
        "scope": "research.read",
        "active": True
    }
}


In [13]:
def validate_token(token: str):
    token_data = ISSUED_TOKENS.get(token)
    
    assert token_data, "invalid token"
    assert token_data["active"], "inactive token"
    return token_data

In [14]:
def require_scope(token_data, required_scope: str):
    scopes = token_data["scope"].split()
    
    assert required_scope in scopes, "insufficient scope"

In [15]:
def access_research_data(token: str):
    token_data = validate_token(token)
    require_scope(token_data, "research.read")
    
    return "This is reasearch data"

In [16]:
access_research_data("token-abc")

'This is reasearch data'

In [17]:
access_research_data("fake-token")


AssertionError: invalid token

In [18]:
ISSUED_TOKENS["token-abc"]["scope"] = "profile.read"
access_research_data("token-abc")


AssertionError: insufficient scope

## Resource Server Summary

- Tokens are the only input
- Issuer decides trust
- Scopes decide authority
- Clients are irrelevant
- Simplicity = security
